# Math genealogy project

MIT License

Copyright (c) 2023
Authors: [Venkata Sai Narayana Bavisetty](https://www.linkedin.com/in/venkata-sai-narayana-bavisetty-62814476/), [Karthik Vasu](https://www.linkedin.com/in/karthikvasu/)



In this notebook, we outline the process of downloading math genealogy data and coordinates for different universities. We will conclude by analyzing the data and providing explanations for some of our findings.


# Downloading math genealogy data

In this notebook, we outline the process of downloading Math genealogy data and coordinates for different universities. We will conclude by analyzing the data and providing explanations for some of our findings.

We utilized the [MGP API](https://mathgenealogy.org:8000/api/v2/MGP/) for data retrieval. Below is the script code for downloading the data. While the script is primarily sourced from the [MGP website](https://mathgenealogy.org:8000/login), we made modifications to implement exponential backoff in case of request failures. This adaptation is crucial for successfully downloading the entire dataset in one go.



We download data in batches of $100$ IDs. Although the website indicates that we can download $10,000$ IDs at once, we encountered difficulties when attempting to do so in batches of $10,000$ or even $1,000$.

In [1]:
#pip install backoff

In [2]:
import requests
import json
import sys
import backoff
from getpass import getpass
import csv
from tqdm import tqdm
from folium import plugins
import folium
from datetime import datetime

In [3]:
PROTOCOL = "https"
HOSTNAME = "mathgenealogy.org"
PORT = "8000"

# Function to prompt on the console for the user's
# email address and password to use to log in to
# the MGP API. Password is not shown on console
# while the user types it.
def getlogin():
    print("Enter email used for MGP authentication:",end=" ",file=sys.stderr)
    email = input()
    password = getpass()
    return {'email': email, 'password': password}

# Function to log in to the MGP API and get a JWT for authentication.
# authdata is a dict with key email set to the user's email address
# and key password set to the user's password.
# If login is successful, returns a JSON object with key token set
# to the JWT.
# If login is unsuccessful, raises RuntimeError.
def login(authdata):
    r = requests.post(f"{PROTOCOL}://{HOSTNAME}:{PORT}/login", authdata)
    if r.ok:
        r.close()
        return r.json()
    else:
        r.close()
        raise RuntimeError("Failed to authenticate")

# Function to do a query against the MGP API. Returns a string with
# the query result if the query was successfully executed. Raises
# RuntimeError if there is an error.

# endpoint is a string (beginning with /) such as "/api/v2/MGP/acad/all"
# referring to an API endpoint

# token is a dict with key token containing a JWT.
# The return value from login() is the best way to get this.

# params is a dict structured to contain the GET parameters.
# For example, if quering /api/v2/MGP/acad, params could be
# {'id': '1969'}.
@backoff.on_exception(backoff.expo, (requests.exceptions.Timeout,requests.exceptions.RequestException), max_tries= 8)
def doquery(endpoint,token, params):
    headers = {'x-access-token': token['token']}
    r = requests.get(f"{PROTOCOL}://{HOSTNAME}:{PORT}{endpoint}",headers = headers, params = params)
    if r.ok:
        r.close()
        return r.text

download_data = False

# User modifications can be made here in this portion of the script.
# Alternatively, import the functions above into your own python
# script and write code similar to what is given here as the example.
if download_data:
    data=[]

    authdata = getlogin()

    # Get all ids from MGP
    endpoint = '/api/v2/MGP/acad/all'
    querydata = ''
    token = login(authdata)
    all_ids = json.loads(doquery(endpoint,token,querydata))
    
    # Download data for IDs
    # We will download in batches of 100
    max_id= int(max(all_ids)/100) +1
    endpoint = '/api/v2/MGP/acad/range'
    json_file = "MGP_data_try.json"

    for id in range(max_id):
        # The token changes every two hours
        token = login(authdata)

        querydata = {'start': id*100, "stop":(id+1)*100}
        new_data = json.loads(doquery(endpoint,token,querydata))
        data.extend(new_data)

        # Save data to a JSON file
        with open(json_file, 'w', encoding='utf-8') as file:
            json.dump(data, file, indent=4, ensure_ascii=False)



# Processing the data

The original data is formatted as a JSON file. Each row is a dictionary featuring the key `MGP_academic`. The corresponding value for this key is another dictionary encompassing the details of individuals within the math genealogy dataset. A sample element is presented below.

In [4]:
import pandas as pd

# Read the JSON file
with open('.\\data\\MGP_data.json', 'r') as file:
    all_data = json.load(file)

print(f"The number of records are {len(all_data)}")
print(f"Here is a sample element")
print(json.dumps(all_data[100], indent=4))

The number of records are 299624
Here is a sample element
{
    "MGP_academic": {
        "ID": "304941",
        "family_name": "Scheepens",
        "given_name": "Roeland",
        "mrauth_id": "",
        "other_names": "Johannus",
        "student_data": {
            "degrees": [
                {
                    "advised by": {
                        "85417": "van de Wetering, Huub M. M.",
                        "97935": "van Wijk, Jarke J."
                    },
                    "degree_msc": "Unknown",
                    "degree_type": "Ph.D.",
                    "degree_year": "2015",
                    "schools": [
                        "Technische Universiteit Eindhoven, Netherlands"
                    ],
                    "thesis_title": "Visualization for maritime situational awareness"
                }
            ],
            "descendants": {
                "advisees": [
                    ""
                ],
                "descendant_count": 0

Here we access the JSON file and convert it into a Pandas dataframe with the required information. The data frame has the columns `ID`,  `family_name`, `given_name`, `school`, `year`, `descendants`, `country` and `subject`. However, for some of these fields may be `None` when information is not available.

In [8]:
data=[]

school_split_len = []
for temp_row in all_data:
    row = temp_row['MGP_academic']

    try:
        school = row['student_data']['degrees'][0]['schools'][0]
    except:
        school = None
    """
    if school is None:
        country = None
    else:
        split = school.split(',')
        school_split_len.append(len(split))
        if len(split)>1:
            country = split[-1]
        school = split[0]
    """
    try:
        year = int(row['student_data']['degrees'][0]['degree_year'])
    except:
        year = None
    
    try:
        subject = int(row['student_data']['degrees'][0]['degree_msc'])
    except:
        subject = None



    new_row = {
        'ID':row['ID'],
        #'family_name':row['family_name'],
        #'given_name':row['given_name'],
        'school':school,
        'year':year,
        'descendants':row['student_data']['descendants']['descendant_count'],
        #'country':country,
        #'subject':subject
        }
    data.append(new_row)

data = pd.DataFrame(data)

In [9]:
data.shape

(299624, 4)

## Exploratory data analysis

Number of PhDs by year plot

Number of PhDs by subject bar plot? or pie chart?

List of people with most descendants by subject classification

Number of people with no descendants you graduated in 2012. This could be correlated to people leaving for industry?

Compare that to people with no desecendants in the past. Show the number of people with no descendants vs graduation year graph.

Compare the number of people staying in the country of their PhD as professors vs people moving out. Check specifically for US to India over time

# Downloading the coordinate data

In [5]:
def get_coordinates(university_name,use_Bing=False,bing_api_key=None):

    # Bing API geocoder
    if use_Bing:
        geolocator = Bing(api_key=bing_api_key)

    # Nominatim geocoder
    else:
        geolocator = Nominatim(user_agent="university_locator")

    university_name_no_filler = remove_fillers(university_name)

    try:
        # Use the geocode method to get the location information
        location = geolocator.geocode(university_name_no_filler)

        if location:
            # Extract latitude and longitude
            latitude, longitude = location.latitude, location.longitude
            return [university_name,latitude,longitude]

        else:
            # if we did not get the location return location in Antartica
            return [university_name,-82,135]
            #print(f"Coordinates for {university_name} not found.")

    except Exception as e:
        #print(f"An error occurred: {e}")
        #If there is an error we return None
        return [university_name,None,None]

def remove_fillers(sentence):

    sentence = str(sentence)
    # List of filler words
    filler_words = ['the', 'of', 'and', 'in', 'to', 'a', 'is', 'that', 'it', 'with', 'as', 'on', 'for', 'at']

    # Split the sentence into words
    words = sentence.split()

    # Remove filler words
    filtered_words = [word for word in words if word.lower() not in filler_words]

    # Join the remaining words to form the new sentence
    new_sentence = ' '.join(filtered_words)

    return new_sentence

In [6]:
import os

# Set download_coordinates to True if you want to download the coordinates
download_coordinates = False

# Path to the coordinates file if you have already downloaded it.
coords_file = '.\\data\\new_coords.csv'

# If you want to use Bing maps API, set use_Bing to True and enter your API key
use_Bing = False
#bing_api_key = os.getenv('BING_API_KEY')
downloaded_file = "data/new_coords"

if download_coordinates:

    import time

    # Using Bing maps API for better performance. This requires a free account and an API key
    from geopy.geocoders import Bing

    # Alternatively we can use OpenStreetMaps API which is free and open source, but does not perform as well as Bing
    from geopy.geocoders import Nominatim

    # Get the unique schools
    schools = data['school'].unique()

    # Loop through the schools and save the recieved coordinates in coord
    coords =[]
    count = 0
    for school in tqdm(schools):
        coord = get_coordinates(school,use_Bing=use_Bing,bing_api_key=bing_api_key)
        coords.append(coord)
        count+=1

        # Save coordinates every 500 iterations
        if count%500==0:
            with open(downloaded_file+str(count)+'.csv','w', newline='') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerows(coords)

        # To prevent the API from blocking us
        time.sleep(0.2)

    # Save all the coordinates
    with open(downloaded_file+'.csv','w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['school','lat','lon'])
        csv_writer.writerows(coords)
    print("Done Downloading Coordinates")

else:
    # Read the coordinates from the csv file
    coords = pd.read_csv(coords_file)


In [10]:
# Merge the coordinates with the data
data = pd.merge(data,coords,on='school',how='left')

# Making heatmaps

In [11]:
print(data.shape)

schools = data['school'].unique()
len(schools)

(299626, 6)


2750

In [40]:
def plot_inf_univ(full_data, t0=1950, t1=1960, ignore_zeroes=True, show_map=True):
    """
    Plots a heatmap of descendants originating from universities
    between years t0 (inclusive) and t1 (exclusive)

    Args:
        full_data: Pandas DataFrame containing descendant data
        t0: Starting year (inclusive)
        t1: Ending year (exclusive)
        ignore_zeroes: Whether to ignore universities with no descendants
        show_map: Whether to display the map

    Returns:
        folium.Map object
    """

    # Suppress chained assignment warnings
    pd.options.mode.chained_assignment = None

    loc = f"Descendants from {t0} to {t1}"
    title_html = f'''
        <h3 align="center" style="font-size:16px"><b>{loc}</b></h3>
        '''

    # Drop NaN values and filter data by year range
    full_data.dropna(inplace=True)
    period = full_data[(full_data["year"] >= t0) & (full_data["year"] < t1)]

    # Group data by university and sum descendants
    p = period[["school", "lat", "lon", "descendants"]].groupby(["school"])
    p = p["descendants"].agg(["sum"]).reset_index()

    # Merge data with coordinates
    p = pd.merge(p, coords, on="school", how="left")

    # Extract relevant data for heatmap
    X = p[["lat", "lon", "sum"]]
    X.dropna(inplace=True)

    # Remove universities with no descendants (optional)
    if ignore_zeroes:
        X = X[X["sum"] != 0]

    # Create map object
    map = folium.Map(location=[5, 30], tiles="Cartodb dark_matter", zoom_start=2)

    # Convert data to format suitable for heatmap plugin
    heat_data = X.values.tolist()

    # Add heatmap layer to map
    plugins.HeatMap(heat_data).add_to(map)

    # Add time period popup to map
    map.get_root().html.add_child(folium.Element(title_html))

    # Save map with appropriate filename based on ignore_zeroes setting
    if ignore_zeroes:
        map.save(f"heat_map_ignore_{t0}_to_{t1}.html")
    else:
        map.save(f"heat_map_{t0}_to_{t1}.html")

    # Display map if requested
    if show_map:
        display(map)

    return map


In [41]:
map = plot_inf_univ(data)


In [50]:
def heatmap_time(full_data, t0=1950, t1=2020, ignore_zeroes=True, show_map=True):
    """
    Generates an animated heatmap illustrating descendant distribution
    across universities for each 5-year period between t0 and t1.

    Args:
        full_data: Pandas DataFrame containing descendant data
        t0: Starting year (inclusive)
        t1: Ending year (exclusive)
        ignore_zeroes: Whether to ignore universities with no descendants
        show_map: Whether to display the map

    Returns:
        folium.Map object
    """
    def update_title(t):
        # Extract the current time from the index
        loc = f"Descendants from {t} to {t+4}"
        title_html = f'''
            <h3 align="center" style="font-size:16px"><b>{loc}</b></h3>
            '''

        # Update the map title based on the current time
        map.get_root().html.find("h3").replace_with(
            folium.Element(title_html)
    )






    # Suppress chained assignment warnings
    pd.options.mode.chained_assignment = None
    
    # Create map object
    map = folium.Map(location=[5, 30], tiles="Cartodb dark_matter", zoom_start=2)

    # Initialize empty list to store heatmap data for each time period
    time_period_data = []

    for t in range(t0, t1, 5):
        loc = f"Descendants from {t0} to {t0+4}"
        title_html = f'''
        <h3 align="center" style="font-size:16px"><b>{loc}</b></h3>
        '''
        # Filter data for the current 5-year period
        period = full_data[(full_data["year"] >= t) & (full_data["year"] < t + 5)]

        # Group data by university and sum descendants
        p = period[["school", "lat", "lon", "descendants"]].groupby(["school"])
        p = p["descendants"].agg(["sum"]).reset_index()

        # Merge data with coordinates
        p = pd.merge(p, coords, on="school", how="left")

        # Extract relevant data for heatmap
        X = p[["lat", "lon", "sum"]]
        X.dropna(inplace=True)

        # Remove universities with no descendants (optional)
        if ignore_zeroes:
            X = X[X["sum"] != 0]

        # Append heatmap data for this time period to the list
        time_period_data.append(X.values.tolist())
        
       

    
    
    

    # Add HeatMapWithTime plugin with data and animation options
    plugins.HeatMapWithTime(time_period_data, auto_play=True, max_opacity=0.3).add_to(map)
    
    # Add time period popup to map
    map.add_callback(update_title(1950))

    # Save map with appropriate filename based on ignore_zeroes setting
    if ignore_zeroes:
        map.save(f"heat_map_ignore_{t0}_to_{t1}.html")
    else:
        map.save(f"heat_map_{t0}_to_{t1}.html")

    if show_map:
        display(map)

    return map


In [51]:
heatmap_time(data, t0 = 1950, t1 = 2020, ignore_zeroes = True, show_map = True)

AttributeError: 'Map' object has no attribute 'add_callback'